# 1. Pre-Setup

## Install Packages

In [ ]:
!pip install roboflow --quiet
!pip install pycocotools --quiet
!pip install albumentations --quiet

## Import Libraries

In [ ]:
import torch
import torchvision
import torchvision.transforms as T
import torch.utils.data
from torch.utils.data import DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.datasets import CocoDetection
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import random
import time
from PIL import Image
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, ConfusionMatrixDisplay
from roboflow import Roboflow
import matplotlib.patches as patches

## Download Dataset from Roboflow

In [ ]:
API_KEY = "4PCfwxj1tbPm25eqqLhX"

rf = Roboflow(api_key=API_KEY)
project = rf.workspace("dent-ydn9e").project("defect-detection-rhju6")
version = project.version(1)
dataset = version.download("coco")
data_dir = dataset.location

print("Downloaded to:", data_dir)

# 2. Hyperparameters and Configs

## Dataset Configs

In [ ]:
train_dir = os.path.join(data_dir, "train")
valid_dir = os.path.join(data_dir, "valid")
test_dir  = os.path.join(data_dir, "test")

## Augmentation Hyperparameters

In [ ]:
augmentation_hyperparameters = {
    'resize':  { 'height': 512, 'width': 512 },
    'jitter': dict(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02, p=0.5),
    'rotate': dict(limit=10, p=0.5, border_mode=0),
    'hflip': dict(p=0.5),
    'vflip': dict(p=0.5),
}

## Dataset Preparation Hyperparameters

In [ ]:
batch_size_train = 4
batch_size_valid = 2
batch_size_test = 2

# Define valid classes to detect: Only use these valid classes (ignore others)
VALID_CLASSES = {
    "dent_marginal": 1,
    "dent_unacceptable": 2
}

## Model Training Hyperparameters

In [ ]:
learning_rate = 0.005
momentum = 0.9
weight_decay = 0.0005

step_size = 3
gamma = 0.1

num_epochs = 10

iou_threshold = 0.5

## Visualization Hyperparameters

In [ ]:
score_threshold = 0.5

# 3. Data Preparation

## Define Dataset Class

In [ ]:
'''
########################################
Coded by: Lee Zhen Ting
########################################
'''

class DefectDetectionDataset(CocoDetection):
    def __init__(self, img_folder, ann_file, transforms=None):
        """
        Custom dataset for defect detection using COCO format annotations.
        Filters out images without relevant class annotations.
        """

        super().__init__(img_folder, ann_file)
        self._transforms = transforms
        self.valid_class_ids = set(VALID_CLASSES.values())

        # Filter out image IDs that do not contain any valid annotations
        self.valid_ids = []
        for idx in range(len(self.ids)):
            anns = self.coco.loadAnns(self.ids[idx])
            if any(ann['category_id'] in self.valid_class_ids for ann in anns):
                self.valid_ids.append(self.ids[idx])

        self.ids = self.valid_ids

    def __getitem__(self, idx):
        img, _ = super().__getitem__(idx)

        # Convert PIL image to numpy array for Albumentations
        img = np.array(img)

        h, w = img.shape[:2]

        ann_ids = self.coco.getAnnIds(imgIds=self.ids[idx])
        anns = self.coco.loadAnns(ann_ids)

        boxes, labels = [], []
        for ann in anns:
            if ann['category_id'] in self.valid_class_ids:
                xmin, ymin, width, height = ann['bbox']

                # Convert COCO bbox (xmin, ymin, width, height) to (xmin, ymin, xmax, ymax)
                xmin = max(0, min(xmin, w-1))
                ymin = max(0, min(ymin, h-1))
                xmax = max(0, min(xmin + width, w-1))
                ymax = max(0, min(ymin + height, h-1))

                # Keep only valid bounding boxes
                if xmax > xmin and ymax > ymin:
                    boxes.append([xmin, ymin, xmax, ymax])
                    labels.append(ann['category_id'])

        # Apply Albumentations transformations if available
        if self._transforms:
            try:
                transformed = self._transforms(
                    image=img,
                    bboxes=boxes,
                    labels=labels
                )
                img = transformed['image']
                boxes = transformed['bboxes']
                labels = transformed['labels']
            except Exception as e:

                # Handle transformation failures gracefully
                print(f"Transform failed for image {self.ids[idx]}: {e}")
                img = self._transforms(image=img)['image']
                boxes = []
                labels = []

        # Convert boxes and labels to tensors
        boxes = torch.as_tensor(boxes, dtype=torch.float32) if boxes else torch.zeros((0, 4))
        labels = torch.as_tensor(labels, dtype=torch.int64) if labels else torch.zeros((0,), dtype=torch.int64)

        # Prepare target dictionary for the model
        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([self.ids[idx]])
        }

        return img, target

## Transform Dataset

### Calculate Mean and Std Function

In [ ]:
'''
########################################
Coded by: Lee Zhen Ting
########################################
'''

def calculate_dataset_stats(dataset, num_samples=None):
    """
    Calculates the mean and standard deviation (per channel) of a dataset's images.

    Args:
        dataset: A PyTorch-style dataset that returns images (in CHW or HWC format).
        num_samples: Number of samples to use for estimation. If None, use the entire dataset.

    Returns:
        mean (list): List of mean values for each channel [R, G, B].
        std (list): List of standard deviation values for each channel [R, G, B].
    """

    if num_samples is None:
        num_samples = len(dataset)
    else:
        num_samples = min(num_samples, len(dataset))

    # Initialize accumulators for sum and squared sum per channel
    pixel_sum = torch.zeros(3)
    pixel_sq_sum = torch.zeros(3)
    num_pixels = 0

    # Randomly select indices to estimate statistics efficiently
    indices = torch.randperm(len(dataset))[:num_samples]

    for idx in tqdm(indices, desc="Calculating stats"):
        img, _ = dataset[int(idx)]

        # Convert image to torch tensor in CHW format if needed
        if isinstance(img, np.ndarray):
            img = torch.from_numpy(img).permute(2, 0, 1)
        elif isinstance(img, torch.Tensor):
            if img.ndim == 3 and img.shape[0] != 3:
                img = img.permute(2, 0, 1)

        # Normalize pixel values to [0, 1]
        img = img.float() / 255.0

        # Accumulate pixel-wise sum and squared sum per channel
        pixel_sum += img.sum(dim=[1, 2])
        pixel_sq_sum += (img**2).sum(dim=[1, 2])
        num_pixels += img.shape[1] * img.shape[2]

    # Compute mean and std per channel
    mean = pixel_sum / num_pixels
    std = torch.sqrt((pixel_sq_sum / num_pixels) - mean**2)

    return mean.tolist(), std.tolist()

### Calculate Mean and Std of Dataset

In [ ]:
'''
########################################
Coded by: Lee Zhen Ting
########################################
'''

# Create a dataset instance without any transforms to ensure raw image statistics
temp_dataset = DefectDetectionDataset(train_dir, os.path.join(train_dir, "_annotations.coco.json"), transforms=None)
mean, std = calculate_dataset_stats(temp_dataset)

print(f"Calculated mean: {mean}")
print(f"Calculated std: {std}")

### Transform Function

In [ ]:
'''
########################################
Coded by: Lee Zhen Ting
########################################
'''

def get_transform(aug=True):
    """
    Returns an Albumentations transform pipeline for training or validation.

    Args:
        aug (bool): If True, returns a training transform with augmentations.
                    If False, returns a validation transform (no augmentation).

    Returns:
        Albumentations Compose object
    """

    if aug:
        # Training pipeline:
        # Resize → Apply augmentations → Normalize → Convert to tensor
        return A.Compose(
            [
                A.Resize(**augmentation_hyperparameters['resize']),
                A.ColorJitter(**augmentation_hyperparameters['jitter']),
                A.Rotate(**augmentation_hyperparameters['rotate']),
                A.HorizontalFlip(**augmentation_hyperparameters['hflip']),
                A.VerticalFlip(**augmentation_hyperparameters['vflip']),
                A.Normalize(mean=mean, std=std),
                ToTensorV2(),
            ],
            bbox_params=A.BboxParams(
                format='pascal_voc',
                label_fields=['labels'],
                clip=True,
                min_visibility=0.3
            )
        )

    # Validation pipeline:
    # Resize → Normalize → Convert to tensor
    return A.Compose(
        [
            A.Resize(**augmentation_hyperparameters['resize']),
            A.Normalize(mean=mean, std=std),
            ToTensorV2(),
        ],
        bbox_params=A.BboxParams(
            format='pascal_voc',
            label_fields=['labels'],
            clip=True,
            min_visibility=0.3
        )
    )

## Load Datasets and DataLoaders

In [ ]:
'''
########################################
Coded by: Lee Zhen Ting
########################################
'''

# True to apply augmentation on train dataset, otherwise false
use_augmentation = True

if use_augmentation:
    train_dataset = DefectDetectionDataset(
        train_dir,
        os.path.join(train_dir, "_annotations.coco.json"),
        transforms=get_transform(aug=True)
    )
else:
    train_dataset = DefectDetectionDataset(
        train_dir,
        os.path.join(train_dir, "_annotations.coco.json"),
        transforms=get_transform(aug=False)
    )

valid_dataset = DefectDetectionDataset(valid_dir, os.path.join(valid_dir, "_annotations.coco.json"), transforms=get_transform(aug=False))
test_dataset  = DefectDetectionDataset(test_dir,  os.path.join(test_dir, "_annotations.coco.json"), transforms=get_transform(aug=False))

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
valid_loader = DataLoader(valid_dataset, batch_size=batch_size_valid, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
test_loader  = DataLoader(test_dataset, batch_size=batch_size_test, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

print(f"Train Samples: {len(train_dataset)}, Valid Samples: {len(valid_dataset)}, Test Samples: {len(test_dataset)}")

# 4. Training and Evaluation Functions

## Training Function

In [ ]:
'''
########################################
Adapted from practical class materials
########################################
'''

def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=50):
    """
    Train the object detection model for one epoch.

    Args:
        model: PyTorch model (e.g., Faster R-CNN).
        optimizer: Optimizer used for updating model weights.
        data_loader: DataLoader providing training data.
        device: Device to train on (CPU or CUDA).
        epoch: Current epoch index (for logging).
        print_freq: How often to print progress.

    Returns:
        float: Average training loss over the epoch.
    """

    model.train()
    running_loss = 0.0

    for batch_idx, (images, targets) in enumerate(data_loader):
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        running_loss += losses.item()

        if (batch_idx + 1) % print_freq == 0:
            print(f"[Epoch {epoch+1}] [Batch {batch_idx+1}/{len(data_loader)}] Training...")

    return running_loss / len(data_loader)

## Evaluation Function

In [ ]:
'''
########################################
Coded by: Goh Ken How
########################################
'''

@torch.no_grad()
def evaluate(model, data_loader, device, epoch=None, print_freq=50):
    """
    Evaluate the model on a validation set by computing loss.
    Note: model.train() is used intentionally to enable loss computation during evaluation,
    as detection models like Faster R-CNN only return losses in training mode.

    Args:
        model: The object detection model.
        data_loader: DataLoader for the validation set.
        device: Computation device (CPU or CUDA).
        epoch: Current epoch (optional, for logging).
        print_freq: How often to log batch progress.

    Returns:
        float: Average validation loss for the epoch.
    """

    # Required to compute loss in models
    model.train()
    running_loss = 0.0

    for batch_idx, (images, targets) in enumerate(data_loader):
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        running_loss += losses.item()

        if (batch_idx + 1) % print_freq == 0:
            if epoch is not None:
                print(f"[Epoch {epoch+1}] [Batch {batch_idx+1}/{len(data_loader)}] Validating...")

    return running_loss / len(data_loader)

## Calculate IOU Function

In [ ]:
'''
########################################
Coded by: Lim Chia Yoong
########################################
'''

def calculate_iou(box1, box2):
    """
    Calculates the Intersection over Union (IoU) between two bounding boxes.

    Args:
        box1: A list or tuple of format [xmin, ymin, xmax, ymax]
        box2: A list or tuple of format [xmin, ymin, xmax, ymax]

    Returns:
        float: IoU value between 0 and 1
    """

    x1, y1 = max(box1[0], box2[0]), max(box1[1], box2[1])
    x2, y2 = min(box1[2], box2[2]), min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)

    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    union_area = box1_area + box2_area - inter_area
    if union_area == 0:
        return 0
    return inter_area / union_area

## Calculate Precision-Recall Function

In [ ]:
'''
########################################
Coded by: Lim Chia Yoong
########################################
'''

def calculate_precision_recall(scores, matches):
    """
    Computes precision and recall values for a list of detection scores and match flags.

    Args:
        scores (list or array): Detection confidence scores.
        matches (list or array): Match indicators (1 for true positive, 0 for false positive).

    Returns:
        precision (np.array): Precision values at each threshold.
        recall (np.array): Recall values at each threshold.
    """

    if len(scores) == 0:
        return np.array([0]), np.array([0])

    scores = np.array(scores)
    matches = np.array(matches)

    indices = np.argsort(-scores)
    matches = matches[indices]

    tp = np.cumsum(matches)
    fp = np.cumsum(1 - matches)

    precision = tp / (tp + fp + 1e-6)
    recall = tp / (tp[-1] + 1e-6)

    return precision, recall

## Calculate mAP Function

In [ ]:
'''
########################################
Coded by: Goh Ken How
########################################
'''

def calculate_map(scores, matches):
    """
    Computes mean Average Precision (mAP) using 11-point interpolation.

    Args:
        scores (list or array): Detection confidence scores.
        matches (list or array): Match indicators (1 = TP, 0 = FP).

    Returns:
        float: Average precision (AP) over 11 recall thresholds.
    """

    precision, recall = calculate_precision_recall(scores, matches)
    ap = 0
    for t in np.linspace(0, 1, 11):
        p = precision[recall >= t]
        if p.size > 0:
            ap += np.max(p)
    return ap / 11

## Compute mAP and PR Curves Function

In [ ]:
'''
########################################
Coded by: Goh Ken How
########################################
'''

@torch.no_grad()
def compute_map_and_pr_curves(model, data_loader, device, iou_threshold=iou_threshold):
    """
    Computes global and per-class mAP and precision-recall curves for a given object detection model.

    Args:
        model: Trained object detection model.
        data_loader: PyTorch DataLoader for the evaluation dataset.
        device: Computation device (e.g., 'cuda' or 'cpu').
        iou_threshold: IoU threshold to consider a prediction as a true positive.

    Returns:
        global_map (float): Mean average precision across all predictions.
        per_class_map (dict): mAP per class {class_id: ap}.
        global_precision (np.array): Precision values for all detections.
        global_recall (np.array): Recall values for all detections.
        per_class_pr (dict): Precision and recall arrays per class {class_id: (precision, recall)}.
    """

    model.eval()

    all_scores = []
    all_matches = []

    # Initialize per-class tracking
    per_class_scores = {c: [] for c in VALID_CLASSES.values()}
    per_class_matches = {c: [] for c in VALID_CLASSES.values()}

    for images, targets in data_loader:
        images = list(img.to(device) for img in images)
        outputs = model(images)

        for output, target in zip(outputs, targets):
            pred_boxes = output['boxes'].cpu().numpy()
            pred_scores = output['scores'].cpu().numpy()
            pred_labels = output['labels'].cpu().numpy()

            gt_boxes = target['boxes'].numpy()
            gt_labels = target['labels'].numpy()

            # Filter out low-confidence predictions
            keep = pred_scores >= 0.05
            pred_boxes = pred_boxes[keep]
            pred_scores = pred_scores[keep]
            pred_labels = pred_labels[keep]

            matched = np.zeros(len(pred_boxes))

            # Global matching (any class)
            for gt_idx, gt_box in enumerate(gt_boxes):
                best_iou, best_idx = 0, -1
                for pred_idx, pred_box in enumerate(pred_boxes):
                    iou = calculate_iou(gt_box, pred_box)
                    if iou > best_iou:
                        best_iou = iou
                        best_idx = pred_idx

                if best_iou >= iou_threshold:
                    matched[best_idx] = 1

            all_scores.extend(pred_scores.tolist())
            all_matches.extend(matched.tolist())

            # Per-class matching
            for c in VALID_CLASSES.values():
                gt_class_boxes = gt_boxes[gt_labels == c]
                pred_class_boxes = pred_boxes[pred_labels == c]
                pred_class_scores = pred_scores[pred_labels == c]

                matched_c = np.zeros(len(pred_class_boxes))

                for gt_idx, gt_box in enumerate(gt_class_boxes):
                    best_iou_c, best_idx_c = 0, -1
                    for pred_idx, pred_box in enumerate(pred_class_boxes):
                        iou = calculate_iou(gt_box, pred_box)
                        if iou > best_iou_c:
                            best_iou_c = iou
                            best_idx_c = pred_idx

                    if best_iou_c >= iou_threshold:
                        if best_idx_c != -1:
                            matched_c[best_idx_c] = 1

                per_class_scores[c].extend(pred_class_scores.tolist())
                per_class_matches[c].extend(matched_c.tolist())

    # Global precision-recall and mAP
    global_precision, global_recall = calculate_precision_recall(all_scores, all_matches)
    global_map = calculate_map(all_scores, all_matches)

    # Per-class precision-recall and mAP
    per_class_pr = {}
    per_class_map = {}
    for c in VALID_CLASSES.values():
        p, r = calculate_precision_recall(per_class_scores[c], per_class_matches[c])
        per_class_pr[c] = (p, r)
        per_class_map[c] = calculate_map(per_class_scores[c], per_class_matches[c])

    return global_map, per_class_map, global_precision, global_recall, per_class_pr

# 5. Model Training

## Model Preparation

### Custom ROI Head with Dropout for Faster R-CNN

In [ ]:
'''
########################################
Coded by: Lim Chia Yoong
########################################
'''

class FastRCNNPredictorWithDropout(nn.Module):
    """
    Custom classification and bounding box regression head for Faster R-CNN
    with a dropout layer added for regularization.

    Args:
        in_channels (int): Number of input features.
        num_classes (int): Number of target classes (including background).
        dropout_prob (float): Dropout probability to apply before prediction.
    """

    def __init__(self, in_channels, num_classes, dropout_prob):
        super().__init__()
        self.dropout = nn.Dropout(dropout_prob)
        self.cls_score = nn.Linear(in_channels, num_classes)
        self.bbox_pred = nn.Linear(in_channels, num_classes * 4)

    def forward(self, x):
        """
        Forward pass through the dropout and two prediction heads.

        Args:
            x (Tensor): Feature vector from ROI pooling (shape: [N, in_channels]).

        Returns:
            scores (Tensor): Class scores for each ROI.
            bbox_deltas (Tensor): Bounding box regression values for each ROI.
        """

        x = self.dropout(x)
        scores = self.cls_score(x)
        bbox_deltas = self.bbox_pred(x)
        return scores, bbox_deltas

### Configurable Faster R-CNN Loader with Optional Dropout and Fine-Tuning

In [ ]:
'''
########################################
Coded by: Goh Ken How
########################################
'''

def get_model(
    num_classes,
    weight_path=None,
    use_dropout=False,
    dropout_prob=0.5,
    freeze_backbone=True,
    unfreeze_layer="layer4"
):
    """
    Loads and customizes a Faster R-CNN model with options for dropout and fine-tuning.

    Args:
        num_classes (int): Number of classes including background.
        weight_path (str, optional): Path to pretrained weights to load.
        use_dropout (bool): Whether to use dropout in the predictor head.
        dropout_prob (float): Dropout rate (if use_dropout is True).
        freeze_backbone (bool): Whether to freeze all backbone layers initially.
        unfreeze_layer (str): Name of layer to unfreeze (e.g., "layer4").

    Returns:
        model (torch.nn.Module): Configured Faster R-CNN model.
    """

    # Load model (with or without pretrained COCO weights)
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)

    # Replace the head to match custom class count
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    if use_dropout:
        model.roi_heads.box_predictor = FastRCNNPredictorWithDropout(in_features, num_classes, dropout_prob)
    else:
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # Load custom-trained weights if provided
    if weight_path:
        model.load_state_dict(torch.load(weight_path, map_location="cpu"))

    # Optional: Freeze entire backbone
    if freeze_backbone:
        for param in model.backbone.body.parameters():
            param.requires_grad = False

    # Optional: Unfreeze a specific layer (e.g., 'layer4') in the backbone
    if unfreeze_layer:
        for name, param in model.backbone.body.named_parameters():
            if unfreeze_layer in name:
                param.requires_grad = True

    return model

### Initialize Model

In [ ]:
'''
########################################
Coded by: Goh Ken How
########################################
'''

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = get_model(num_classes=3)
model.to(device)

## Training Loop

In [ ]:
'''
########################################
Coded by: Lim Chia Yoong
########################################
'''

# Initialize training state
start_epoch = 0
best_map = 0.0

# Set up optimizer and learning rate scheduler
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

# Lists to store training history
train_losses = []
valid_losses = []
maps = []
per_class_maps = {c: [] for c in VALID_CLASSES.values()}

# Load from checkpoint if available
checkpoint_path = "checkpoint.pth"
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    best_map = checkpoint['best_map']
    model.train()
    print(f"Loaded checkpoint from epoch {start_epoch-1}, best validation mAP so far: {best_map:.4f}")
else:
    print("No checkpoint found, starting from scratch.")

# Training loop
for epoch in range(start_epoch, num_epochs):
    start_time = time.time()

    train_loss = train_one_epoch(model, optimizer, train_loader, device, epoch)
    valid_loss = evaluate(model, valid_loader, device, epoch)

    global_map, per_class_map, global_precision, global_recall, per_class_pr = compute_map_and_pr_curves(model, valid_loader, device)

    lr_scheduler.step()

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    maps.append(global_map)
    for c in VALID_CLASSES.values():
        per_class_maps[c].append(per_class_map[c])

    elapsed = time.time() - start_time

    print(f"Epoch [{epoch+1}/{num_epochs}] Completed:")
    print(f"    Train Loss        : {train_loss:.4f}")
    print(f"    Valid Loss        : {valid_loss:.4f}")
    print(f"    Validation mAP@0.5: {global_map:.4f}")
    print(f"    Time Elapsed      : {elapsed/60:.2f} min")
    print("-" * 50)

    # Save best model
    if global_map > best_map:
        best_map = global_map
        torch.save(model.state_dict(), "best_fasterrcnn_defect_detection.pth")
        print(f"Best model saved with mAP {best_map:.4f}")

    # Save full checkpoint
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': lr_scheduler.state_dict(),
        'best_map': best_map
    }, f'checkpoint_epoch{epoch+1}.pth')
    print(f"Checkpoint saved at epoch {epoch+1}")

print("Training Complete!")

# 6. Metrics Saving and Visualizations

## Save Training Results to CSV

In [ ]:
'''
########################################
Coded by: Lim Chia Yoong
########################################
'''

metrics_df = pd.DataFrame({
    "Epoch": list(range(1, num_epochs+1)),
    "Train Loss": train_losses,
    "Valid Loss": valid_losses,
    "mAP@0.5": maps
})
metrics_df.to_csv("training_metrics.csv", index=False)
print("Saved training metrics to training_metrics.csv")

## Plot Loss Curves

In [ ]:
'''
########################################
Coded by: Goh Ken How
########################################
'''

plt.figure()
plt.plot(train_losses, label="Train Loss")
plt.plot(valid_losses, label="Valid Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Train and Validation Loss")
plt.grid(True)
plt.savefig("loss_curve.png")
plt.show()

## Plot Overall mAP Curve

In [ ]:
'''
########################################
Coded by: Lee Zhen Ting
########################################
'''

plt.figure()
plt.plot(maps, label="Validation mAP@0.5")
plt.xlabel("Epoch")
plt.ylabel("mAP@0.5")
plt.legend()
plt.title("Validation mAP@0.5 Curve")
plt.grid(True)
plt.savefig("map_curve.png")
plt.show()

## Plot Per-Class mAP Curve

In [ ]:
'''
########################################
Coded by: Lee Zhen Ting
########################################
'''

plt.figure()
for c, values in per_class_maps.items():
    class_name = [k for k,v in VALID_CLASSES.items() if v == c][0]
    plt.plot(values, label=f"{class_name}")
plt.xlabel("Epoch")
plt.ylabel("Per-Class mAP@0.5")
plt.legend()
plt.title("Per-Class Validation mAP@0.5")
plt.grid(True)
plt.savefig("per_class_map_curve.png")
plt.show()

## Plot PR Curves

### Plot PR Curves Function

In [ ]:
'''
########################################
Coded by: Lim Chia Yoong
########################################
'''

def plot_pr_curve(precision, recall, title, filename):
    plt.figure()
    plt.plot(recall, precision, marker='.')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(title)
    plt.grid(True)
    plt.savefig(filename)
    plt.show()

### Plot Global PR Curve

In [ ]:
'''
########################################
Coded by: Lim Chia Yoong
########################################
'''

plot_pr_curve(global_precision, global_recall, "Global PR Curve", "global_pr_curve.png")

### Plot Per-Class PR Curves

In [ ]:
'''
########################################
Coded by: Lim Chia Yoong
########################################
'''

for class_id, (precision_c, recall_c) in per_class_pr.items():
    class_name = [k for k,v in VALID_CLASSES.items() if v == class_id][0]
    plot_pr_curve(precision_c, recall_c, f"{class_name} PR Curve", f"{class_name.lower()}_pr_curve.png")

## Visualize Predictions

In [ ]:
'''
########################################
Coded by: Goh Ken How
########################################
'''

model.eval()
for i in range(5):
    img, _ = test_dataset[i]
    img = img.to(device)
    with torch.no_grad():
        prediction = model([img])

    img = img.permute(1,2,0).cpu().numpy()

    fig, ax = plt.subplots(1, 1, figsize=(12,9))
    ax.imshow(img)
    boxes = prediction[0]['boxes'].cpu().numpy()
    scores = prediction[0]['scores'].cpu().numpy()
    labels = prediction[0]['labels'].cpu().numpy()

    for box, score, label in zip(boxes, scores, labels):
        if score >= score_threshold:
            xmin, ymin, xmax, ymax = box
            width, height = xmax - xmin, ymax - ymin
            rect = patches.Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            label_name = [k for k,v in VALID_CLASSES.items() if v == label][0]
            ax.text(xmin, ymin, f"{label_name}: {score:.2f}", color='white', fontsize=12, backgroundcolor='red')

    plt.axis('off')
    plt.savefig(f"prediction_{i}.png")
    plt.show()